In [ ]:
import os

# Compatibility cell: enable eager execution early
import tensorflow as tf
try:
    tf.config.run_functions_eagerly(True)
except Exception:
    pass
print('eager:', tf.executing_eagerly())

In [2]:
%pwd

'd:\\End_to_End_ML_project_Chiken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End_to_End_ML_project_Chiken_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

# Enable eager execution to fix numpy() error
tf.compat.v1.enable_eager_execution()

[2025-12-19 19:13:27,091: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2025-12-19 19:13:27,099: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [21]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        # Ensure functions run eagerly before loading
        import tensorflow as tf
        tf.config.run_functions_eagerly(True)
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        # Compile the model to ensure it's trainable
        # Use categorical loss because model outputs 2-class probabilities
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    def train_valid_generator(self):

        # Use tf.data API for compatibility with eager execution
        import tensorflow as tf
        self.train_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.20,
            subset="training",
            seed=123,
            image_size=tuple(self.config.params_image_size[:-1]),
            batch_size=self.config.params_batch_size,
            label_mode='int'  # produce integer labels, we'll one-hot them to match model output
        )

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.20,
            subset="validation",
            seed=123,
            image_size=tuple(self.config.params_image_size[:-1]),
            batch_size=self.config.params_batch_size,
            label_mode='int'
        )

        # Normalize images
        normalization_layer = tf.keras.layers.Rescaling(1./255)
        self.train_generator = self.train_generator.map(lambda x, y: (normalization_layer(x), y))
        self.valid_generator = self.valid_generator.map(lambda x, y: (normalization_layer(x), y))

        # Convert integer labels to one-hot vectors matching model output
        num_classes = int(self.model.output_shape[-1]) if self.model is not None else 2
        self.train_generator = self.train_generator.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=num_classes)))
        self.valid_generator = self.valid_generator.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=num_classes)))

        # Optional augmentation
        if self.config.params_is_augmentation:
            data_augmentation = tf.keras.Sequential([
                tf.keras.layers.RandomFlip("horizontal"),
                tf.keras.layers.RandomRotation(0.1),
                tf.keras.layers.RandomZoom(0.1),
            ])
            self.train_generator = self.train_generator.map(lambda x, y: (data_augmentation(x, training=True), y))

    @staticmethod
    def save_model(path: Path, model: 'tf.keras.Model'):
        model.save(path)


    def train(self, callback_list: list):
        # Use Dataset directly with Keras
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [22]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-12-19 19:49:16,887: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-19 19:49:16,890: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-19 19:49:16,892: INFO: common: created directory at: artifacts]
[2025-12-19 19:49:16,893: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-12-19 19:49:16,895: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-12-19 19:49:16,896: INFO: common: created directory at: artifacts\training]
[2025-12-19 19:49:17,229: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 390 files belonging to 2 classes.
Using 312 files for training.
Found 390 files belonging to 2 classes.
Using 78 files for validation.
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.6839 - loss: 0.7348 [2025-12-19 19:53:20,235: WARNING: sav

In [23]:
# Diagnostic cell: inspect model loss, output shape and dataset label shapes
config = ConfigurationManager()
training_config = config.get_training_config()
training = Training(config=training_config)
training.get_base_model()
print('model.loss:', training.model.loss)
print('model.output_shape:', training.model.output_shape)
training.train_valid_generator()
import tensorflow as tf
train_iter = iter(training.train_generator)
x_batch, y_batch = next(train_iter)
print('x_batch.shape:', getattr(x_batch, 'shape', None))
print('y_batch.shape:', getattr(y_batch, 'shape', None))
print('y_batch.dtype:', y_batch.dtype)
print('y_batch sample:', y_batch[0])


[2025-12-19 19:54:15,402: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-19 19:54:15,407: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-19 19:54:15,409: INFO: common: created directory at: artifacts]
[2025-12-19 19:54:15,412: INFO: common: created directory at: artifacts\training]
[2025-12-19 19:54:15,786: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
model.loss: categorical_crossentropy
model.output_shape: (None, 2)
Found 390 files belonging to 2 classes.
Using 312 files for training.
Found 390 files belonging to 2 classes.
Using 78 files for validation.
x_batch.shape: (16, 224, 224, 3)
y_batch.shape: (16, 2)
y_batch.dtype: <dtype: 'float32'>
y_batch sample: tf.Tensor([0. 1.], shape=(2,), dtype=float32)
